# Goal of laboratory work
The purpose of this work is to implement an algorithm for calculating the degree of closeness of objects described by a set of numerically expressed properties using CUDA technology.

# Task Definition
We have a set of objects that can be described by sets of properties expressed numerically. In this task, it is necessary to calculate the degree of similarity of objects. As input parameters we have a matrix:

![](images/CropperScreenshot_2019-06-21_21:35:23.png)

It is necessary to calculate the distance between each possible pair of rows of the matrix.

![](images/CropperScreenshot_2019-06-21_21:35:41.png)

The paper assumes the implementation of this algorithm using the software-hardware architecture of parallel computing CUDA with global and shared memory. It is necessary to determine the kernel configuration and perform acceleration tests with various examples of input parameters.

# Brief Theory
CUDA technology introduces a number of additional extensions for the C language, which are necessary for writing code for the GPU:
1. Function specifiers that show how and where functions will be performed from;
2. The variable specifiers that serve to indicate the type of memory used by the GPU;
3. GPU kernel launch qualifiers;
4. Built-in variables for identifying threads, blocks and other parameters when executing code in the GPU core.
5. Additional types of variables.


Function specifiers determine how and from where the functions will be called. There are 3 such specifiers in CUDA:
- __host__ 
- __global__ 
- __device__ 


Kernel launch qualifiers are used to describe the number of blocks, threads, and memory that need to be allocated when calculating on a GPU. 
The CUDA host API is the link between the CPU and the GPU.


# ALGORITHM (METHOD) of IMPLEMENTATION

In [1]:
%cat mm.cu

#include <cuda_runtime.h>

#include <iostream>
#include <memory>
#include <string>

#include <cuda.h>
#include <stdio.h>




#ifndef BLOCK_SIZE
# define BLOCK_SIZE 16
#endif

#ifndef _M
# define _M 10000
#endif

#ifndef _N
# define _N 10000
#endif

#if !defined(CUDA) && !defined(CPU) && !defined(CHECK)
# define CUDA
#endif

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"gpuAssert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

__global__ void mx_dist(float *m_in, float *m_out, int m, int n) 
{
    int i = blockIdx.y * blockDim.y + threadIdx.y; 
    int j = blockIdx.x * blockDim.x + threadIdx.x;
	float s = 0, sum = 0;

    if( i < m && j < m) {

    	for(int k = 0; k < n; ++k) {
    		s = m_in[i*m + k] - m_in[j*m + k];
    		sum += s*s;
    	}

    	// printf("--> %d %d %f %f\n", j, i, m

# Result And Experiments

In [2]:
import subprocess
import os

def compile(*defs, **defskw):
    args = [f"-D{k}" for k in defs] + [f"-D{k}={v}" for k, v in defskw.items()]
    _cmd = 'nvcc mm.cu -o mm'.split() + args
    # print(' '.join(_cmd))
    cmd = subprocess.run(_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  
    if(cmd.stdout): print('cmd.stdout', cmd.stdout)
    if(cmd.stderr): print('cmd.stderr', cmd.stderr)
    
def run(env=None):
    cmd = subprocess.run('./mm', stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=env)
    return cmd.stdout.decode('utf8')    




Lets see how the performance on CPU compares with the performace on GPU

In [3]:
compile('CPU', _N=10_00, _M=50_0)
print("Execution time on CPU", end="\n\t")
%timeit run()
print()

compile('CUDA', _N=10_00, _M=50_0)
print("Execution time on GPU", end="\n\t")
%timeit run()
print()

Execution time on CPU
	1.2 s ± 371 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Execution time on GPU
	19.5 ms ± 394 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



We see a clear gain of onGPU execution.

Lets analyze how the block size affects performance.

In [4]:

for bs in [8, 16, 32, 64, 128, 256, 512, 1024, 2048]:
    compile('CUDA', _N=10_00, _M=50_0, BLOCK_SIZE=bs)
    print(f"Execution time with block size {bs}", end="\n\t")
    %timeit run()
    print()



Execution time with block size 8
	29.7 ms ± 698 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 16
	21.6 ms ± 2.95 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Execution time with block size 32
	28.4 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 64
	21.2 ms ± 3.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 128
	21.5 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Execution time with block size 256
	21.8 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 512
	24.2 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 1024
	22.2 ms ± 940 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Execution time with block size 2048
	22.6 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



# Conclusion

In this paper, we implemented an algorithm for calculating the degree of proximity of objects described by a set of numerically expressed properties using CUDA technology. Tests were performed for the kernel with global and shared memory.
As a result of this work, the skill of working with CUDA technology of writing code for the GPU was obtained.